# STAGE5 하이퍼 파라미터 튜닝

## 하이퍼 파라미터란?
**초매개변수**(hyper parameter)는 학습모델의 학습과정을 제어하기위한 선택사항입니다. 

예시) 학습률(learning rate), 손실함수(cost function), 미니배치크기(mini-batch size), 가중치 초기화(weight initialization) 등

<br/>

학습모델마다 다른 초매개변수가 필요하고, 간단한 선형회귀 모델같은 경우는 초매개변수가 없기도 합니다.

초매개변수를 어떻게 설정하느냐에 따라 학습시간이 달라지고, 대부분 과적합, 과소적합을 초래합니다. 

그렇기 때문에 강한 모델을 위한 **초매개변수 최적화**는 필수적입니다.

<br/>

하지만 초매개변수 최적화는 휴리스틱(heuristic)한 방법이나

경험상 통용되는(rules of thumb)방법에 의해 결정되는 경우가 많습니다. 

😓 "학습모델을 처음 사용하는 데... 어떡하나요?"

여러 초매개변수를 바꾸며 학습하는 많은 시도를 해야합니다!

<br/>
 
저희 팀은 여러 시도를 도와주는 방법들을 실습했습니다.
- Grid Search
- Random Search
- Bayesian
- Optuna
- Pycaret

함께한 모델은 랜덤포레스트와 LGBM입니다.

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier 
from lightgbm import LGBMClassifier

#### RF
저희는 모델링에서 랜덤포레스트의 초매개변수를 설정하지 않은 모델을 학습했습니다.

초매개변수 최적화를 위해 랜덤포레스트의 주요 변수를 알아보겠습니다. (과적합 조절은 대게 과소적합 조절을 포함합니다.)

랜덤포레스트는 트리기반의 앙상블 분류 모델입니다.
- 의사결정나무의 구조: 뿌리에서 잎까지 **아래로** 향하는 구조입니다.
- 뿌리(root) -> 분할노드(split node, decision node) -> 잎사귀노드(leaf node)

<br/>

#### 주요 하이퍼 파라미터

#### 1. criterion ({'gini','entropy','log_loss, default='gini'}

모델의 분류평가지표를 설정하는 변수이며, 특정 알고리즘을 기준으로 분류를 실시합니다.

<br/>

#### 2. n_estimators (int, default: 100) -> 성능 및 시간 조절

n_estimators는 트리의 개수를 조절하는 변수입니다.

분류모델의 개수가 과도하게 적거나 많다면 문제가 발생합니다.

<br/>

#### 3. max_depth (int, default= None) -> 과적합 조절

깊이를 조절할 수 있는 변수입니다.

트리기반 분류모델은 위(root)에서 아래(leaf)으로 "분류의 진행상태"를 깊이(depth)라고 표현합니다.

특정(잘못된) 방향으로 부족하게 또는 과하게 진행되었다면 문제가 발생합니다. 

<br/>

#### 4. min_samples_split (int or float, default= 2) -> 과적합 조절

노드를 분할하는 최소한의 샘플 데이터 수입니다.

아직 분류되지 않은 샘플이 지정된 개수 이상의 데이터가 남았다면 분류를 하는 것이죠! 

과도하게 쪼개다보면 과적합이 발생합니다. 

<br/>

#### 5. min_samples_leaf (int or float, default= 1) -> 과적합 조절


min_sample_split과 함께 과적합을 조절할 수있지만, 조금 다릅니다. 

min_sample_split의 경우 split 전에 확인하는 것이고,
min_sample_leaf의 경우 split 후에 확인할 수 있는 것입니다. 

그러므로 하나의 잎으로 분류된 객체가 설정된 개수(min_sample_leaf)보다 적다면 분류가 되지 않는 것으로 과적합을 조절합니다.  

<br/>

#### 6. max_features ({'sqrt', 'log2', None}, default= "sqrt")

최적의 분류를 위한 feature의 개수를 산출하는 식을 조절하는 변수입니다.

자세한 사항은 아래 공식문서를 참고해주세요.

<br/>
 
#### 7. max_leaf_nodes (default= None) -> 과적합 조절

잎사귀노드의 최대 개수입니다.

잎사귀노드의 수가 부족하거나 과하다면 문제가 발생합니다. 

<br/>

---
참고문헌: 

- Scikit-learn의 RandomForest 공식문서입니다.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html



#### LGBM
저희는 모델링과정에서 Light GBM 또한 튜닝없이 진행했습니다. 

LGBM은 큰 데이터셋에 활용도가 높습니다. 특히 학습시간을 정말 많이 단축시킬 수 있습니다!

<br/>

주요 하이퍼파라미터
#### 1. learning_rate (double, default=0.1)

학습률입니다. gradient descent의 학습량을 조절합니다.

<br/>

#### 2. max_depth (int, default=-1)

<= 0 means no limit

랜덤포레스트와 동일합니다.

<br/>

#### 3. num_iterations (int, default=100)

랜덤포레스트의 n_estimators와 동일합니다.

<br/>

#### 4. boosting ({'gbdt','rf','dart'}, default='gbdt')

부스팅 방법입니다.

'gbdt'는 GBM입니다. 'rf'는 랜덤포레스트입니다. 'dart'는 dropout을 이용한 앙상블모델입니다.

dart의 경우 다른 분류기보다 높은 정확도를 형성한다고 알려져있습니다.

추가적인 정보는 dart의 논문을 참고해주세요. (참고문헌 확인)

<br/>

#### 5. metrics

학습 목적에 따른 평가지표입니다.

binary(cross entropy), muticlass(cross entrop), regression_l1(mae), regression_l2(mse), mape(mape)를 포함하여 여러 지표를 지원합니다. 

자세한 사항은 아래 공식문서를 참고해주세요.

<br/>

#### 6. early_stopping_round (int, default=0)

<= 0 means disable

정확도가 더이상 향상되지 않을 때 조기에 멈추게 되는 반복횟수를 지정합니다. 

<br/>

#### 7.  bagging_fraction (float, default=1.0)

행단위로 샘플링합니다. 과적합을 방지할 수 있습니다.

<br/>

#### 8. feature_fraction (float, default=1.0)

열단위로 샘플링합니다. 과적합을 방지할 수 있습니다.

<br/>

#### 9. scale_pos_weight ()

불균형 데이터의 postive의 가중치를 높혀준다. 민감한 파라미터입니다! 불균형 데이터에 효과적입니다.

<br/>

---

참고문헌:

- LightGBM의 공식문서입니다.

https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

- DART의 논문입니다.

Vinayak, R. K., & Gilad-Bachrach, R. (2015, February). Dart: Dropouts meet multiple additive regression trees. In Artificial Intelligence and Statistics (pp. 489-497). PMLR.

## 튜닝 방법

### Grid Search

그리드서치는 초매개변수 최적화의 가장 기본적인 방법입니다.

학습 전에 지정한 초매개변수의 집합들을 모두 시행하여 가장 우수한 초매개변수 집합을 얻습니다.

<br/>

---
참고문헌:

- Sklearn의 GridSearchCv 공식문서입니다.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

- scoring에 관한 공식문서입니다.

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer

In [ ]:
rf = RandomForestClassifier()

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4, 6, 8, None],
    'max_features': ['sqrt','log2'],
    'n_estimators': [10, 30, 70, 100]
}

In [ ]:
grid_search = GridSearchCV(
    estimator=rf, # model
    param_grid=param_grid, # set the grid
    scoring=make_scorer(accuracy_score), # 평가지표, 추가정보 -> 참고문헌확인
    cv=10, # STAGE 6 cross validation
)
grid_search.fit(train_x, train_y)

GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rand

In [ ]:
grid_search.predict(valid_x)
grid_search_pred = grid_search.predict_proba(valid_x)

In [ ]:
print("Best parameters found: ", grid_search.best_params_)

Best parameters found:  {'criterion': 'gini', 'max_depth': 4, 'max_features': 'log2', 'n_estimators': 10}


In [ ]:
print(f'정확도: {accuracy_score(valid_y, grid_search.predict(valid_x)) * 100:.4f}%')
print(f'재현율: {recall_score(valid_y, grid_search.predict(valid_x),pos_label=1) * 100:.4f}%')
print(f'F1_score: {f1_score(valid_y, grid_search.predict(valid_x)) * 100:.4f}%')
print(f'ROC_AUC: {roc_auc_score(valid_y, grid_search.predict(valid_x)) * 100:.4f}%')

정확도: 94.4444%
재현율: 100.0000%
F1_score: 97.1429%
ROC_AUC: 50.0000%


### Random Search

랜덤서치는 지정된 범위 내에서 무작위로 초매개변수를 조합하여 최적화를 시도합니다.

<br/>


장단점은 **무작위**하다는 점입니다. 

경우에 따라 그리드서치에 비해 빠르고 느릴 수 있습니다. 

또한 성과를 가늠할 수 없기때문에 생각치 못한 결과를 맞이할 수 있습니다. 

그렇기 때문에 최적화 보다 성과의 방향성을 살펴보기 위해 주로 시도합니다.

<br/>

---
참고문헌:

"랜덤서치가 그리드서치보다 효율적임"을 주장하는 논문도 있습니다. 

[Bergstra, J., & Bengio, Y. (2012). Random search for hyper-parameter optimization. Journal of machine learning research, 13(2).](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model = RandomForestClassifier()

param_distributions = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4, 6, 8, None],
    'max_features': ['sqrt','log2'],
    'n_estimators': [10, 30, 70, 100]
}
randomized_search = RandomizedSearchCV(model, 
                                       param_distributions=param_distributions, 
                                       n_iter=50, cv=10, # STAGE 6 cross validation
                                       scoring='accuracy', return_train_score=True,
                                       verbose=1,
                                       random_state=random_state)

randomized_search.fit(train_x, train_y)

df = pd.DataFrame(randomized_search.cv_results_).sort_values(by=['mean_test_score', 'mean_train_score'], ascending=False)
display(df[['params', 'mean_train_score', 'mean_test_score']].head(10))

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.1min finished


,params,mean_train_score,mean_test_score
1,"{'n_estimators': 100, 'max_features': 'sqrt', ...",1.0,0.951905
2,"{'n_estimators': 70, 'max_features': 'log2', '...",1.0,0.951905
5,"{'n_estimators': 100, 'max_features': 'log2', ...",1.0,0.951905
6,"{'n_estimators': 70, 'max_features': 'sqrt', '...",1.0,0.951905
15,"{'n_estimators': 70, 'max_features': 'sqrt', '...",1.0,0.951905
19,"{'n_estimators': 70, 'max_features': 'sqrt', '...",1.0,0.951905
21,"{'n_estimators': 70, 'max_features': 'log2', '...",1.0,0.951905
23,"{'n_estimators': 100, 'max_features': 'log2', ...",1.0,0.951905
39,"{'n_estimators': 100, 'max_features': 'log2', ...",1.0,0.951905
44,"{'n_estimators': 100, 'max_features': 'sqrt', ...",1.0,0.951905


In [ ]:
print('최적 하이퍼파라미터: ', randomized_search.best_params_)
print('학습 검증 정확도:', randomized_search.best_score_)

print(f'정확도: {accuracy_score(valid_y, randomized_search.predict(valid_x)) * 100:.4f}%')
print(f'재현율: {recall_score(valid_y, randomized_search.predict(valid_x),pos_label=1) * 100:.4f}%')
print(f'F1_score: {f1_score(valid_y, randomized_search.predict(valid_x)) * 100:.4f}%')
print(f'ROC_AUC: {roc_auc_score(valid_y, randomized_search.predict(valid_x)) * 100:.4f}%')

최적 하이퍼파라미터:  {'n_estimators': 70, 'max_features': 'sqrt', 'max_depth': 4, 'criterion': 'entropy'}
학습 검증 정확도: 0.9519047619047617
정확도: 94.4444%
재현율: 100.0000%
F1_score: 97.1429%
ROC_AUC: 50.0000%


### Bayesian Optimization

**베이지안 추론**이란?

베이지안 추론(Bayesian inference)은 통계적 추론으로, 사건에 대한 사전(prior)확률과 추가적인 정보에의한 사건에 대한 사후(posterior)확률을 입니다.

<br/>

**베이지안 최적화**란?

사전 정보를 최적 값 탐색에 반영하는 것이라고 이해할 수 있습니다.

함수f(x)을 최대로 만드는 최적해를 찾는 것을 목적으로 하는 최적화 방법입니다.

베이지안 최적화 모델은 Surrogate Model(대체 모델, 근사수학모델, ), Acquisition Function의 두가지 요소가 반드시 필요합니다.

<br/>


**Surrogate Model**이란? 

뜻은 대체모델 또는 근사수학모델입니다. 복잡한 시스템의 수많은 입출력 특성으로 실제 모형과 유사하게 만드는 모델입니다. 

베이지안 최적화 모델은 확률적인 추정을 통해 최적화를 시도합니다.

예를 들면, 자동차 충돌실험모델 등이 있습니다.

<br/>

**Acquisition Function** 이란? 

최적해를 찾는 과정에서 가장 유용할 다음 입력값의 후보를 추천해 주는 함수입니다.

수학적으로 목적함수에 대해서 확률적 추정을 통한 모델을 기반으로 다음 탐색지점을 결정합니다.

이때 정보량이 많은 곳을 우선적으로 추천하게 됩니다.

요약하면 최적의 값을 추천하는 함수입니다.

<br/>


---

참고문헌:

Bayesina optimization의 github입니다.

https://github.com/fmfn/BayesianOptimization

In [ ]:
# pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
def lgbm_evaluate(learning_rate, max_depth, n_estimators, bagging_fraction, feature_fraction, scale_pos_weight):
    model = LGBMClassifier(random_state=random_state,
                           objective='binary',
                           metric='binary_logloss',
                           learning_rate=learning_rate,
                           max_depth=int(max_depth),
                           n_estimators=int(n_estimators),
                           bagging_fraction=float(bagging_fraction),
                           feature_fraction=float(feature_fraction),
                           scale_pos_weight=float(scale_pos_weight)
    )
    accuracy = np.mean(cross_val_score(model, train_x, train_y, cv=10, scoring=make_scorer(accuracy_score))) # STAGE 6 cross validation

    return accuracy

In [ ]:
tuned_lgbm = BayesianOptimization(lgbm_evaluate, 
                                  {"learning_rate": (0.001, 0.1),
                                   "max_depth": (-1, 20),
                                   "n_estimators": (100,10000),
                                   "bagging_fraction": (0.3, 0.7),
                                   "feature_fraction": (0.3, 0.7),
                                   "scale_pos_weight": (0.01, 0.15)
})

tuned_lgbm.maximize(init_points=5, n_iter=10, acq='ei',) # init_points: 임의탐색횟수, n_iter: 주변 탐색, acq: 제공하는 함수 선택

|   iter    |  target   | baggin... | featur... | learni... | max_depth | n_esti... | scale_... |
-------------------------------------------------------------------------------------------------


<ipython-input-242-5af6982f19ef>:10: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  tuned_lgbm.maximize(init_points=5, n_iter=10, acq='ei',) # init_points: 임의탐색횟수, n_iter: 주변 탐색, acq: 제공하는 함수 선택


| 1         | 0.9329    | 0.6802    | 0.4813    | 0.008052  | -0.9236   | 4.964e+03 | 0.1414    |
| 2         | 0.9376    | 0.5889    | 0.5726    | 0.0485    | 19.33     | 6.567e+03 | 0.1038    |
| 3         | 0.9376    | 0.5659    | 0.3453    | 0.04337   | 14.71     | 2.103e+03 | 0.05452   |
| 4         | 0.9233    | 0.6015    | 0.4268    | 0.08452   | 1.25      | 872.2     | 0.1222    |
| 5         | 0.9329    | 0.3913    | 0.4691    | 0.03177   | 17.7      | 6.472e+03 | 0.1162    |
| 6         | 0.9329    | 0.4903    | 0.3869    | 0.05464   | 15.74     | 6.672e+03 | 0.05081   |
| 7         | 0.9329    | 0.5574    | 0.5061    | 0.01281   | 16.01     | 2.102e+03 | 0.05561   |
| 8         | 0.9376    | 0.636     | 0.6487    | 0.05558   | 18.51     | 8.477e+03 | 0.05636   |
| 9         | 0.9329    | 0.6903    | 0.5604    | 0.07796   | 4.237     | 3.375e+03 | 0.01645   |
| 10        | 0.9281    | 0.5736    | 0.5983    | 0.07075   | 0.3786    | 590.7     | 0.07174   |
| 11        | 0.9329

In [ ]:
tuned_lgbm.max['params']

{'bagging_fraction': 0.5889330482450694,
 'feature_fraction': 0.5725915935375597,
 'learning_rate': 0.048499029875590356,
 'max_depth': 19.325383056784766,
 'n_estimators': 6566.964373017123,
 'scale_pos_weight': 0.10381324415995875}

### Optuna

Optuna는 초매개변수 최적화 프레임워크 입니다.

optuna는 가볍고 효율적으로 초매개변수 최적화를 지원합니다. 또한 함수식이 직관적이며 시각화도 편리합니다.

<br/>

옵튜나는 두 단어(study, trial)를 사용합니다.

**Study**란?

objective function에 기반한 일련의 최적화 과정입니다.

<br/>

**Trial**란?

objective function의 1번의 시행

study의 목표는 n번의 trial를 통해 최적의 초매개변수 조합을 찾는 것입니다.

<br/>

---
참고문헌:
- 자세한 사항은 아래 Oputna의 Github를 참고해주세요.

https://github.com/optuna/optuna 

- 아래는 optuna의 예시 목록입니다.

https://github.com/optuna/optuna-examples

In [ ]:
# optuna 설치하는 방법
# pip install optuna

In [ ]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler # oputna의 시행방법은 TPESampler, SKoptsampler 등이 있다.

In [ ]:
def objective(trial: Trial) -> float: # 목적함수를 정의한다.
    global train_x, train_y, valid_x, valid_y
    params_lgb = {
        "random_state": 0,
        "verbosity": -1,
        "objective": "binary", # 종속변수 = open, close
        "metric": "binary_logloss",
        
        "learning_rate": trial.suggest_float("reg_alpha", 0.001, 0.1),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "n_estimators": 10000,
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.3, 0.7),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.3, 0.7),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.01, 0.15),
    }
    model = LGBMClassifier(**params_lgb) # params_lgb에 입력한 함수를 모두 받아온다. "**"는 입력값을 길이제한 없이 받아온다.
    model.fit(train_x,train_y,
        eval_set=[(train_x, train_y), (valid_x, valid_y)],
        early_stopping_rounds=500, # early_stopping은 500번으로 제한했다.
        verbose=True, # 진행사항 on/off
    )
    lgb_pred = model.predict_proba(valid_x)
    log_score = log_loss(valid_y, lgb_pred) # log_loss를 사용한다.
    return log_score 

In [ ]:
sampler = TPESampler(seed=0)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize", # 목적함수의 return은 log_score이다. 즉, log_loss를 최소화하는 study
    sampler=sampler,      # 만일 accuracy라든지 roc-auc 같이 값이 커져야 하는 경우는 maximize로 설정해야 합니다.
)                         
study.optimize(objective, n_trials=100,) # trial을 100번 시행
print("Best Score:", study.best_value) # 최상의 성적을 가져온다.
print("Best trial:", study.best_trial.params) # 최상의 컨디션을 가진 변수집합을 가져온다.

In [ ]:
# Lgbm(base)
lgbm = LGBMClassifier(random_state=random_state)
lgbm.fit(train_x, train_y)
# tuned Lgbm
tuned_lgbm = LGBMClassifier(**study.best_trial.params) # 튜닝된 하이퍼파라미터를 받아온다.
tuned_lgbm.fit(train_x, train_y)

LGBMClassifier(bagging_fraction=0.6503981375433736,
               feature_fraction=0.6196769255303484, max_depth=4,
               reg_alpha=0.01940607436602854,
               scale_pos_weight=0.11936722683048441)

In [ ]:
for model, name in zip([lgbm, tuned_lgbm], ['lgbm','tuned_lgbm']):
    print('##########',name,'##########')
    print(f'정밀도: {metrics.precision_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'재현율: {recall_score(valid_y, model.predict(valid_x),pos_label=1) * 100:.4f}%')
    print(f'f1_score: {f1_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'정확도: {accuracy_score(valid_y, model.predict(valid_x)) * 100:.4f}%')
    print(f'ROC_AUC: {roc_auc_score(valid_y, model.predict(valid_x)) * 100:.4f}%')

########## lgbm ##########
정밀도: 94.3820%
재현율: 98.8235%
f1_score: 96.5517%
정확도: 93.3333%
ROC_AUC: 49.4118%
########## tuned_lgbm ##########
정밀도: 94.3820%
재현율: 98.8235%
f1_score: 96.5517%
정확도: 93.3333%
ROC_AUC: 49.4118%


😥 안타깝지만, 역시나 튜닝의 결과와 기본모델와 같네요.. 

더 큰 데이터셋을 이용하시길 바랍니다!

### AutoML by pycaret

In [ ]:
lightgbm = create_model('lightgbm')
tuned_lightgbm = tune_model(lightgbm)
rf = create_model('rf')
tuned_rf = tune_model(rf)

# 마무리

이번 Stage에서 배운 다양한 방법을 활용하여 데이터를 전처리하는 방법은 다른 대회에서도 유용하게 응용할 수 있을 것입니다.

따라서 ‘병원 개/폐업 분류 예측 경진대회🏥’뿐만 아니라 다른 주제의 대회에서도 이번 stage가 큰 도움이 될 것이라 기대합니다. 😉

데이커 여러분 잘 따라오셨나요. 그럼 다음 stage에서 뵙겠습니다.. 🎉